In [3]:
import os
from secret import OPENAI_API_KEY

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
from langchain import OpenAI 
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [41]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

<h2>Standard Tool</h2>

In [26]:
!pip install --upgrade --quiet  duckduckgo-search -q

In [27]:
from langchain.tools import DuckDuckGoSearchResults
from langchain.agents import Tool
from langchain.tools import BaseTool

search = DuckDuckGoSearchResults()
# defining a single tool
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

<h2>Custom Tool|</h2>

In [7]:
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

In [21]:
from langchain.agents import Tool
from langchain.tools import BaseTool

life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be Meaning Of Life "
)

In [10]:
import random

def random_num(input=""):
    return random.randint(0,5)

In [11]:
random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

<h2>Creating Agent</h2>

In [33]:
from langchain.agents import initialize_agent

tools = [search, random_tool, life_tool]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [34]:
conversational_agent("Who is the president of Bangladesh?")



> Entering new AgentExecutor chain...
```json
{
    "action": "duckduckgo_results_json",
    "action_input": "President of Bangladesh"
}
```
Observation: [snippet: 24 Apr 2023. Shahabuddin Chuppu, a retired judge, has been sworn in as the new president of Bangladesh for a five-year term, replacing Abdul Hamid. Parliamentary Speaker Shirin Sharmin Choudhury ..., title: Shahabuddin Chuppu takes oath as Bangladesh's 22nd president - Al Jazeera, link: https://www.aljazeera.com/news/2023/4/24/shahabuddin-chuppu-takes-oath-as-bangladeshs-22nd-president], [snippet: Mohammad Sahabuddin, a veteran freedom fighter and politician, was sworn in as the 22nd President of Bangladesh on April 24, 2023. He was elected by Parliament and succeeded Abdul Hamid, who completed his term in office., title: Mohammad Sahabuddin sworn in as 22nd President of Bangladesh, link: https://www.bssnews.net/news/president/122139], [snippet: The freedom fighter and former judge succeeds Md Abdul Hamid, who resigned on 

{'input': 'Who is the president of Bangladesh?',
 'chat_history': [],
 'output': 'Shahabuddin Chuppu is the current President of Bangladesh, having been sworn in for a five-year term on April 24, 2023, replacing Abdul Hamid.'}

In [35]:
conversational_agent("Can you give me a random number?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Random number",
    "action_input": "random"
}
```
Observation: 0
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The response to your last comment was 0."
}
```

> Finished chain.


{'input': 'Can you give me a random number?',
 'chat_history': [HumanMessage(content='Who is the president of Bangladesh?'),
  AIMessage(content='Shahabuddin Chuppu is the current President of Bangladesh, having been sworn in for a five-year term on April 24, 2023, replacing Abdul Hamid.')],
 'output': 'The response to your last comment was 0.'}

In [37]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The meaning of life is a philosophical question that has been debated for centuries. Different cultures, religions, and individuals have varying beliefs and interpretations about the purpose and significance of life. Some believe it is about seeking happiness and fulfillment, others see it as a journey of spiritual growth and enlightenment, while some view it as a quest for knowledge and understanding. Ultimately, the meaning of life is a deeply personal and subjective concept that each individual may interpret in their own way."
}
```

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='Who is the president of Bangladesh?'),
  AIMessage(content='Shahabuddin Chuppu is the current President of Bangladesh, having been sworn in for a five-year term on April 24, 2023, replacing Abdul Hamid.'),
  HumanMessage(content='Can you give me a random number?'),
  AIMessage(content='The response to your last comment was 0.'),
  HumanMessage(content='What is the meaning of life?'),
  AIMessage(content='The response to your last comment was 0.')],
 'output': 'The meaning of life is a philosophical question that has been debated for centuries. Different cultures, religions, and individuals have varying beliefs and interpretations about the purpose and significance of life. Some believe it is about seeking happiness and fulfillment, others see it as a journey of spiritual growth and enlightenment, while some view it as a quest for knowledge and understanding. Ultimately, the meaning of life is a deeply per

In [38]:
# system prompt
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template

'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system that can help with a wide range 

In [39]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''



conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [40]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Meaning of Life",
    "action_input": "Meaning Of Life"
}
```
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded but is actually 42.17658"
}
```

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='Can you give me a random number?'),
  AIMessage(content='The response to your last comment was 0.'),
  HumanMessage(content='What is the meaning of life?'),
  AIMessage(content='The response to your last comment was 0.'),
  HumanMessage(content='What is the meaning of life?'),
  AIMessage(content='The meaning of life is a philosophical question that has been debated for centuries. Different cultures, religions, and individuals have varying beliefs and interpretations about the purpose and significance of life. Some believe it is about seeking happiness and fulfillment, others see it as a journey of spiritual growth and enlightenment, while some view it as a quest for knowledge and understanding. Ultimately, the meaning of life is a deeply personal and subjective concept that each individual may interpret in their own way.')],
 'output': 'The meaning of life is 42 if rounded but is actually 42.17658'}